# Lesson 4
## New .env variables
you need to add three new variables to you r .env file, if you didnt have done so yet

```env
HOSPITAL_AGENT_MODEL=gpt-3.5-turbo-1106
HOSPITAL_CYPHER_MODEL=gpt-3.5-turbo-1106
HOSPITAL_QA_MODEL=gpt-3.5-turbo-0125
```
Your .env file now includes variables that specify which LLM you’ll use for different components of your chatbot. You’ve specified these models as environment variables so that you can easily switch between different OpenAI models without changing any code. Keep in mind, however, that each LLM might benefit from a unique prompting strategy, so you might need to modify your prompts if you plan on using a different suite of LLMs.

## RAG search.
In this part we will create a Document Chain +LLM Chain that implement RAG to manage que queries about the reviews, We will use Neo4j as the vector database.
### Vector Search Indexes
Vector databases are specifically designed to store and query high-dimensional vectors, which are numerical representations of unstructured data like text, images, or audio. Vector search indexes enable efficient similarity searches (semantic searches) based on that semantically similar objects are represented by vectors that are close to each other in the vector space.

Neo4j, a graph database, has added vector search capabilities as an add-on. Neo4j's [Vector search indexes](https://neo4j.com/docs/cypher-manual/current/indexes-for-vector-search/) allows developers to create vector indexes. These indexes work with node or relationship properties that contain vector embeddings

### Creating the embeddings
In LangChain, you can use Neo4jVector to create review embeddings and returns a "vector store" to query it. Here’s the code to create the indexes.

The method takes several parameters:
- embedding: an instance of OpenAIEmbeddings, which will be used to generate embeddings for the text data. The specific embedding model used will depend on the requirements of your project, such as the type of text data, the desired level of accuracy, and the computational resources available. We use it as we have openai api key but there are others in the langchain library.
    - HuggingFaceEmbeddings: uses Hugging Face's Transformers library to generate embeddings
    - SentenceTransformersEmbeddings: uses the Sentence Transformers library to generate embeddings
    - CustomEmbeddings: a custom implementation of the Embeddings interface, allowing you to use your own embedding model
    - Word2VecEmbeddings
    - GloVeEmbeddings
    - FastTextEmbeddings
    - BERTEmbeddings
- url, username, password: these are used to connect to the Neo4j database, and are retrieved from environment variables (NEO4J_URI, NEO4J_USERNAME, NEO4J_PASSWORD).
- index_name: the name of the vector index to create ("reviews").
- node_label: the label of the nodes in the graph database that will be indexed ("Review").
- text_node_properties: a list of properties on the nodes that contain text data to be indexed (["physician_name", "patient_name", "text", "hospital_name"]).
- embedding_node_property: the property on the nodes where the generated embeddings will be stored ("embedding").

---
> ⚠️ **Note:** `from_existing_graph` is primarily used when your text data is already in the Neo4j graph, but the embeddings and the vector index might not yet exist or need to be generated by the LangChain process. It seems to automate querying Neo4j to get the relevant text data, calculate embeddings for that text, storing those newly generated embeddings back into the specified node property in Neo4j and creating the necessary vector index if it doesn't exist. Therefore, if the embeddings and the corresponding vector index already fully exist and are populated in your Neo4j database, the method you would typically use is `Neo4jVector.from_existing_index`, not `from_existing_graph`
---



In [ ]:
import os
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain_openai import OpenAIEmbeddings


neo4j_vector_index = Neo4jVector.from_existing_graph(
    embedding=OpenAIEmbeddings(),
    url=os.getenv("NEO4J_URI"),
    username=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD"),
    index_name="reviews",
    node_label="Review",
    text_node_properties=[
        "physician_name",
        "patient_name",
        "text",
        "hospital_name",
    ],
    embedding_node_property="embedding",
)

In [2]:
neo4j_vector_index

If you filled the database in the previous lesson, after running this you will see that the embeddings had been created as an index of the review text 

![embeddings.png](embeddings.png)

This allows you to answer questions like "Which hospitals have had positive reviews?" It also allows the LLM to tell you which patient and physician wrote reviews matching your question.

### The retriever
To use the Retrieval Augmented Generation we need to create a retriever based in this vector store we just created.

In [3]:

from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)
HOSPITAL_QA_MODEL = os.getenv("HOSPITAL_QA_MODEL") # gpt-3.5-turbo-0125

# chat prompt template with system and human messages
review_template = """Your job is to use patient
reviews to answer questions about their experience at a hospital. Use
the following context to answer questions. Be as detailed as possible, but
don't make up any information that's not from the context. If you don't know
an answer, say you don't know.
{context}
"""

review_system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(input_variables=["context"], template=review_template)
)

review_human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(input_variables=["question"], template="{question}")
)
messages = [review_system_prompt, review_human_prompt]

review_prompt = ChatPromptTemplate(
    input_variables=["context", "question"], messages=messages
)

# the document chain, based in our vector index
reviews_vector_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model=HOSPITAL_QA_MODEL, temperature=0),
    chain_type="stuff",
    retriever=neo4j_vector_index.as_retriever(k=12),
)

# the complete review chain
reviews_vector_chain.combine_documents_chain.llm_chain.prompt = review_prompt

The last part is setting up a RetrievalQA chain, what is called a document chain.

**The retriever/document chain**

* `reviews_vector_chain` is the instance of `RetrievalQA`
* The `from_chain_type` defines the chain type (pass all the docs found entirely, pass only relevant chunks of it, ...). In this case, the chain type is `"stuff"` the standard one, it passes all retrieved data as context. Different chain types are "stuff", "map refine", and "rerank", with the later are alternative complex ones to avoid overflowing the context window 
* The `llm` parameter specifies the language model that will be used specifically for the question-answering system and its temperature.
* The `retriever` parameter specifies the vector store (retriever) that will be used to fish the relevant documents from the database. In this case, it's an instance of `neo4j_vector_index.as_retriever(k=12)`, which is a retriever that uses the Neo4j vector index to gather documents. The `k` parameter specifies the number of documents to retrieve (12).

**Combining the documents chain and LLM chain**

* The `combine_documents_chain` method is used to combine the documents chain (which retrieves relevant documents from the database) with the LLM chain (which uses the language model to generate answers).
* The `llm_chain.prompt` parameter specifies the prompt that will be used for the LLM chain. In this case, it's set to the `review_prompt` instance that was defined earlier.


In [5]:
query = """What have patients said about hospital efficiency?
        Mention details from specific reviews."""

response = reviews_vector_chain.invoke(query)
response.get("result")

Patients have generally praised the efficiency of the hospital staff and the cleanliness of the facilities. For example, Kevin Cox mentioned that the hospital staff at Wallace-Hamilton was efficient, and Jesse Marquez also noted that the hospital provided excellent medical care at the same facility. Additionally, Kim Franklin commented on the efficiency of the hospital staff and the cleanliness of the facilities at Wallace-Hamilton.


## DB query search.
In this part we will create the Cypher generation chain that you’ll use to answer queries about structured hospital system data.

Neo4j Cypher chain will accept a user’s natural language query, convert the natural language query to a Cypher query, run the Cypher query in Neo4j, and use the Cypher query results to respond to the user’s query. You’ll leverage LangChain’s ```GraphCypherQAChain``` for this. 

If we were to use a normal SQL database we would use langchain's own ```SQLDatabase``` instead ```Neo4jGraph``` & ```SQLDatabaseChain``` instead ```GraphCypherQAChain```

### DB Connection neo4j-style
To get started we need first to have a LLM capable database connection with ```Neo4jGraph```:

In [1]:
import os
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph(
    url=os.getenv("NEO4J_URI"),
    username=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD"),
)

graph.refresh_schema()

The Neo4jGraph object is a LangChain wrapper that allows LLMs to execute queries on your Neo4j instance. You instantiate graph using your Neo4j credentials, and you call graph.refresh_schema() to sync any recent changes to your instance.

### cypher generation template. 
We want the LLM to generate a Cypher (neo4j query language) query based in a human prompt.

This needs heavy prompting engineering and serious trial and error:
- we need the answer to provide the query and only the query
- It should only query, not insert, update or delete
- we rather it use alias to make the query readable for debug purposes
- we need to inform beforehand of the equivalent in the DB of certain status, businmess particulars that are not self evident from the schema i.e: 
    - visit is concluded if has a discharge date
    - payment was rejected/pending if it does not have the service_date in the COVERED BY relationship
    - Use abbreviations when filtering on hospital states (e.g. "Texas" is "TX", "Colorado" is "CO", "North Carolina" is "NC",...)
    - list some categorical values of a few node properties for it to know beforehand
    - etc.
    
All of the detail you provide in your prompt template improves the LLM’s chance of generating a correct Cypher query for a given question. If you’re curious about how necessary all this detail is, try creating your own prompt template with as few details as possible. Then run questions through your Cypher chain and see whether it correctly generates Cypher queries.

In [2]:
from langchain.prompts import PromptTemplate

cypher_generation_template = """
Task:
Generate Cypher query for a Neo4j graph database.

Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.

Schema:
{schema}

Note:
Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything other than
for you to construct a Cypher statement. Do not include any text except
the generated Cypher statement. Make sure the direction of the relationship is
correct in your queries. Make sure you alias both entities and relationships
properly. Do not run any queries that would add to or delete from
the database. Make sure to alias all statements that follow as with
statement (e.g. WITH v as visit, c.billing_amount as billing_amount)
If you need to divide numbers, make sure to
filter the denominator to be non zero.

Examples:
# Who is the oldest patient and how old are they?
MATCH (p:Patient)
RETURN p.name AS oldest_patient,
       duration.between(date(p.dob), date()).years AS age
ORDER BY age DESC
LIMIT 1

# Which physician has billed the least to Cigna
MATCH (p:Payer)<-[c:COVERED_BY]-(v:Visit)-[t:TREATS]-(phy:Physician)
WHERE p.name = 'Cigna'
RETURN phy.name AS physician_name, SUM(c.billing_amount) AS total_billed
ORDER BY total_billed
LIMIT 1

# Which state had the largest percent increase in Cigna visits
# from 2022 to 2023?
MATCH (h:Hospital)<-[:AT]-(v:Visit)-[:COVERED_BY]->(p:Payer)
WHERE p.name = 'Cigna' AND v.admission_date >= '2022-01-01' AND
v.admission_date < '2024-01-01'
WITH h.state_name AS state, COUNT(v) AS visit_count,
     SUM(CASE WHEN v.admission_date >= '2022-01-01' AND
     v.admission_date < '2023-01-01' THEN 1 ELSE 0 END) AS count_2022,
     SUM(CASE WHEN v.admission_date >= '2023-01-01' AND
     v.admission_date < '2024-01-01' THEN 1 ELSE 0 END) AS count_2023
WITH state, visit_count, count_2022, count_2023,
     (toFloat(count_2023) - toFloat(count_2022)) / toFloat(count_2022) * 100
     AS percent_increase
RETURN state, percent_increase
ORDER BY percent_increase DESC
LIMIT 1

# How many non-emergency patients in North Carolina have written reviews?
MATCH (r:Review)<-[:WRITES]-(v:Visit)-[:AT]->(h:Hospital)
WHERE h.state_name = 'NC' and v.admission_type <> 'Emergency'
RETURN count(*)

String category values:
Test results are one of: 'Inconclusive', 'Normal', 'Abnormal'
Visit statuses are one of: 'OPEN', 'DISCHARGED'
Admission Types are one of: 'Elective', 'Emergency', 'Urgent'
Payer names are one of: 'Cigna', 'Blue Cross', 'UnitedHealthcare', 'Medicare',
'Aetna'

A visit is considered open if its status is 'OPEN' and the discharge date is
missing.
Use abbreviations when
filtering on hospital states (e.g. "Texas" is "TX",
"Colorado" is "CO", "North Carolina" is "NC",
"Florida" is "FL", "Georgia" is "GA", etc.)

Make sure to use IS NULL or IS NOT NULL when analyzing missing properties.
Never return embedding properties in your queries. You must never include the
statement "GROUP BY" in your query. Make sure to alias all statements that
follow as with statement (e.g. WITH v as visit, c.billing_amount as
billing_amount)
If you need to divide numbers, make sure to filter the denominator to be non
zero.

The question is:
{question}
"""

cypher_generation_prompt = PromptTemplate(
    input_variables=["schema", "question"], template=cypher_generation_template
)


---
> ⚠️ **Note:** The above prompt template provides the LLM with four examples of valid Cypher queries for your graph. Giving the LLM a few examples and then asking it to perform a task is known as few-shot prompting, and it’s a simple yet powerful technique for improving generation accuracy. However, few-shot prompting might not be sufficient for Cypher query generation, especially if you have a complicated graph. One way to improve this is to create a vector database that embeds example user questions/queries and stores their corresponding Cypher queries as metadata. When a user asks a question, you inject Cypher queries from semantically similar questions into the prompt, providing the LLM with the most relevant examples needed to answer the current question.
---

### cypher results interpretation template
Next, you define the prompt template for the question-answer component of your chain. This template tells the LLM to use the Cypher query results from launching the query obtained with the previous prompt to generate a nicely-formatted answer to the user’s query. Again heavy prompting required, but not quite. 

In [3]:
qa_generation_template = """You are an assistant that takes the results
from a Neo4j Cypher query and forms a human-readable response. The
query results section contains the results of a Cypher query that was
generated based on a user's natural language question. The provided
information is authoritative, you must never doubt it or try to use
your internal knowledge to correct it. Make the answer sound like a
response to the question.

Query Results:
{context}

Question:
{question}

If the provided information is empty, say you don't know the answer.
Empty information looks like this: []

If the information is not empty, you must provide an answer using the
results. If the question involves a time duration, assume the query
results are in units of days unless otherwise specified.

When names are provided in the query results, such as hospital names,
beware  of any names that have commas or other punctuation in them.
For instance, 'Jones, Brown and Murray' is a single hospital name,
not multiple hospitals. Make sure you return any list of names in
a way that isn't ambiguous and allows someone to tell what the full
names are.

Never say you don't have the right information if there is data in
the query results. Always use the data in the query results.

Helpful Answer:
"""

qa_generation_prompt = PromptTemplate(
    input_variables=["context", "question"], template=qa_generation_template
)

### The GraphCypherQAChain
This chain will integrate both parts: the creation of the query and the parse of the results retrieved by launching it.

Here’s a breakdown of the parameters used in GraphCypherQAChain.from_llm():

- cypher_llm: The LLM used to generate Cypher queries.
- qa_llm: The LLM used to generate an answer given Cypher query results.
- graph: The Neo4jGraph object that connects to your Neo4j instance. The database connection if you will.
- verbose: Whether intermediate steps your chain performs should be printed.
- qa_prompt: The prompt template for responding to questions/queries.
- cypher_prompt: The prompt template for generating Cypher queries.
- validate_cypher: If true, the Cypher query will be inspected for errors and corrected before running. Note that this doesn’t guarantee the Cypher query will be valid. Instead, it corrects simple syntax errors that are easily detectable using regular expressions.
- top_k: The number of query results to include in qa_prompt.

In [6]:
from langchain_openai import ChatOpenAI
from langchain.chains import GraphCypherQAChain

HOSPITAL_QA_MODEL = os.getenv("HOSPITAL_QA_MODEL")
HOSPITAL_CYPHER_MODEL = os.getenv("HOSPITAL_CYPHER_MODEL")

hospital_cypher_chain = GraphCypherQAChain.from_llm(
    cypher_llm=ChatOpenAI(model=HOSPITAL_CYPHER_MODEL, temperature=0),
    qa_llm=ChatOpenAI(model=HOSPITAL_QA_MODEL, temperature=0),
    graph=graph,
    verbose=True,
    qa_prompt=qa_generation_prompt,
    cypher_prompt=cypher_generation_prompt,
    validate_cypher=True,
    top_k=100,
)

In [7]:
question = """What is the average visit duration for emergency visits in North Carolina?"""
response = hospital_cypher_chain.invoke(question)
response.get("result")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (v:Visit)-[:AT]->(h:Hospital)
WHERE h.state_name = 'NC' AND v.admission_type = 'Emergency' AND v.status = 'DISCHARGED'
WITH v, duration.between(date(v.admission_date), date(v.discharge_date)).days AS visit_duration
RETURN AVG(visit_duration) AS average_visit_duration
Full Context:
[{'average_visit_duration': 15.072972972972991}]

> Finished chain.


'The average visit duration for emergency visits in North Carolina is approximately 15.07 days.'

In [8]:
question = """Which state had the largest percent increase in Medicaid visits from 2022 to 2023?"""
response = hospital_cypher_chain.invoke(question)
response.get("result")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (h:Hospital)<-[:AT]-(v:Visit)-[:COVERED_BY]->(p:Payer)
WHERE p.name = 'Medicaid' AND v.admission_date >= '2022-01-01' AND
v.admission_date < '2024-01-01'
WITH h.state_name AS state, COUNT(v) AS visit_count,
     SUM(CASE WHEN v.admission_date >= '2022-01-01' AND
     v.admission_date < '2023-01-01' THEN 1 ELSE 0 END) AS count_2022,
     SUM(CASE WHEN v.admission_date >= '2023-01-01' AND
     v.admission_date < '2024-01-01' THEN 1 ELSE 0 END) AS count_2023
WITH state, visit_count, count_2022, count_2023,
     (toFloat(count_2023) - toFloat(count_2022)) / toFloat(count_2022) * 100
     AS percent_increase
RETURN state, percent_increase
ORDER BY percent_increase DESC
LIMIT 1
Full Context:
[{'state': 'TX', 'percent_increase': 8.823529411764707}]

> Finished chain.


'The state with the largest percent increase in Medicaid visits from 2022 to 2023 was Texas (TX) with a percent increase of 8.82%.'